## Home Assignment 3

Submit your solution via Moodle until 23.59pm on Wednesday, December 18th. Late submissions are accepted for 12 hours following the deadline, with 1/3 of the total possible points deducted from the score.

Submit your solutions in teams of up 2-3 members. **Single student submissions will not be graded anymore**.
Please denote all members of the team with their student ID and full name in the notebook. In this home assignment, you have to submit an .ipynb notebook and a file "labels.npy" as specified in task 3. Do not submit anything else than these two files!

Cite ALL your sources for coding this home assignment. In case of plagiarism (copying solutions from other teams or from the internet) ALL team members will be expelled from the course without warning.

##### List team members, including all student IDs here:
1. Student 1 (123456)
2. Student 2 (123457)
3. (optional) Student 3 (123458)

In [14]:
import numpy as np
import pandas as pd

### Task 1: Fair Classification with the Two Naive Bayes Algorithm (2 pts)

Recall the _Ricci_ dataset that we have discussed in lecture, which shows the results of an exam for promotion eligibility. For the purpose of this exercise, we want to build a classifier which achieves that the group that gets a promoted (_Class_ attribute) is fairly balanced with respect to the sensitive attribute *Race* - as noted in lecture, enforcing this equality on that data has actually caused a "reverse discrimination" lawsuit. The majority class, i.e. the white population, has sensitive attribute value 1, and all non-whites have sensitive attribute value 0.

The dataset is read in the cell below, and an implementation of the CV measure can also be found below that. Note that in the following tasks, you should make use of functions from scikit-learn whereever possible.

In [16]:
df = pd.read_csv("ricci.csv")
df.head()


,Oral,Written,Race,Combine,Class,Position_Captain,Position_Lieutenant
0,89.52,95,1,92.808,1,1,0
1,80.00,95,1,89.000,1,1,0
2,82.38,87,1,85.152,1,1,0
3,88.57,76,1,81.028,1,1,0
4,76.19,84,1,80.876,1,1,0


In [3]:
def cv_measure(y_pred, s_arr):
    pos_mask = s_arr==1
    neg_mask = s_arr==0

    return 1.0 - np.sum(y_pred[pos_mask])/np.sum(pos_mask) + np.sum(y_pred[neg_mask])/np.sum(neg_mask)

#### a) Biased Classification (0.5 pts)

Train a Gaussian Naive Bayes classifier on the full dataset which learns to predict the _Class_ attribute using all other columns in the data. Apply your model on the training data, and compute the CV score of the resulting predictions with respect to the sensitive attribute _Race_. What does the result say on the disparity in the classifications of each majority and minority group?

In [100]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.naive_bayes import GaussianNB

X = df.drop(columns = ['Class'])
y = df['Class']

# pre-prossing of the dataset, scale all measures in [0,1]
scaler = MinMaxScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

# After scale the accuracy of dataset has been gradually improved from 0.47 to 0.91.
print('Accuracy of the dataset without Scale:',clf1.score(X,y))
print('Accuracy of the dataset with Scale:',clf1.score(X_scaled,y))

# Train a Gaussian Naive Bayes classifier to calculate the CV-Socre
clf1 = GaussianNB()
clf1.fit(X_scaled, y)
y_pred = clf1.predict(X_scaled)
s_arr = X['Race'].to_numpy()

print('CV-Score with original date is:',cv_measure(y,s_arr))
print('CV-Score from GNB classifier:',cv_measure(y_pred,s_arr))


Accuracy of the dataset without Scale: 0.4745762711864407
Accuracy of the dataset with Scale: 0.8389830508474576
CV-Score with original date is: 0.6970588235294117
CV-Score from GNB classifier: 0.5341176470588235


Explaination:</br>

The CV-Score from original dataset is about 0.69. After training a Gaussian naive Bayes classiﬁer from data involving a sensitive feature, the CV score on the predicted classes decrease to about 0.53. That means the difference of  majority and minority group increase from 0.33 to 0.47.</br>

This shows that the non-whites records are more frequently misclassiﬁed to the class 0 than white population records; and thus, the individuals in non-whites with class 1 are considered to be unfairly treated.</br>
(class 0 is the people who are fail in the exam and class 1 is the people who are not fail in the exam)</br>

#### b) Applying Two Naive Bayes Models (0.5 pts)

Train two different Naive Bayes Models, one for each of the two race values. Again, apply these on the data samples that corresponds to the population they were trained on, and combine their predictions. Are the classifications among both population groups more balanced?

In [99]:

# split the data set into two parts, one for the people with race value 0 and the other part with race value 1
df_race_1 = df[df['Race'] == 1]
df_race_0 = df[df['Race'] == 0]

# train the model for df_Race_1 and 

X1 = df_race_1.drop(columns = ['Class'])
y1 = df_race_1['Class']
X2 = df_race_0.drop(columns = ['Class'])
y2 = df_race_0['Class']

# scale the measures of X1 except "Race"
X1_no_Race = X1.drop(columns = ['Race'])
scaler = MinMaxScaler()
scaler.fit(X1_no_Race)
X1_no_Race = scaler.transform(X1_no_Race)
X1_scaled = np.insert(X1_no_Race, 2, values=1, axis=1)

# scale the measures of X2 except "Race"
X2_no_Race = X2.drop(columns = ['Race'])
scaler = MinMaxScaler()
scaler.fit(X2_no_Race)
X2_no_Race =scaler.transform(X2_no_Race)
X2_scaled = np.insert(X2_no_Race, 2, values=0, axis=1)


# train the model for df_Race_1 and df_race_0
clf1 = GaussianNB()
clf2 = GaussianNB()
clf1.fit(X1_scaled, y1)
clf2.fit(X2_scaled, y2)

y1_pred = clf1.predict(X1_scaled)
y2_pred = clf2.predict(X2_scaled)
y_pred = np.concatenate([y1_pred,y2_pred])
y_origin = np.concatenate([y1,y2])
X_2 = np.concatenate([X1_scaled, X2_scaled])
s_arr = X_2[:,2]

print('The CV-Score with original date is:',cv_measure(y_origin,s_arr))
print('The CV-Score after training is:',cv_measure(y_pred,s_arr))


The CV-Score with original date is: 0.6970588235294117
The CV-Score after training is: 0.7811764705882354


Explaination:
Compare with the CV-Score between the dataset, which is before and after training. The CV-Socre isncreases from 0.69 to 0.78. It shows that the classifications among both population groups are more balanced.

#### c) The Two Naive Bayes Algorithm by Calders and Verwer (1 pt)

In lecture we have briefly touched the Two Naive Bayes algorithm by Calders and Verwer. Its main idea is to train two different Naive Bayes models corresponding to both values of a binary sensitive attribute $S$, and to then adapt the prior probabilities $P(Y|S)$ of each of these models until the predictions on the training data become fair.
More precisely, in training, we first fit two different models, namely one for the privileged class $S=1$ and one for the non-privileged class $S=0$, to obtain the conditional probabilities $P(X_i|Y,S=s)$, $s=0,1$, and the priors $P(Y|S=s)$, $s=0,1$ from the training data. After these initial fits, we evaluate the model on the training data, and if we do not observe fair predictions, i.e. predictions for which the CV score is at least $1-\epsilon$ for some $\epsilon>0$, we adapt the priors $P(Y|S=s)$ until this is the case. This adaptation is carried out as follows:

We initialize $N(Y=j, S=s)$ as the number of training samples with class $j$ and sensitive attribute value $s$, and $N(Y=1)$ the number of samples with class $1$ in the data. Further, let $\Delta$ denote an adaptation rate, which is given as input.  
Then we conduct the following iteration:

     cv_score <- CV score of the predicted classes by the current model.
     while cv_score < 1 - \epsilon:
        n_pos <- number of predicted positive samples from the current model.
        if n_pos < N(Y=1):
           N(Y=1, S=0) <- N(Y=1, S=0) + ΔN(Y=0, S=1)
           N(Y=0, S=0) <- N(Y=0, S=0) - ΔN(Y=0, S=1) 
        else:
           N(Y=0, S=1) <- N(Y=0, S=1) + ΔN(Y=1, S=0)
           N(Y=1, S=1) <- N(Y=1, S=1) - ΔN(Y=1, S=0)
        if any of the N(Y, S) is negative then cancel the previous update of N(Y, S) and abort
        Recalculate priors P(Y|S=i)  and cv_score based on updated N(Y, S)
    
Note that the priors $P(Y|S=s)$ are calculated via
$$P(Y|S=s) = \frac{N(Y, S=s)}{N(S=s)},$$
where $N(S=s)$ denotes the number of samples in the data with sensitive attribute value $s$.

**Task description:** Implement a class that runs the Two Naive Bayes algorithm, using the signatures in the cell below! Apply Gaussian Naive Bayes models to fit your data. In particular, you do not need to differentiate between continuous and discrete data as in home assignment 2, and you can use the GaussianNB class from sklearn to fit the two individual models. Note that this class also has an attribute that stores the prior probabilities and can be overwritten easily.
Further, you may assume that all input data is valid, i.e. you do not need to check whether for instance the feature matrix X has as many rows as the class vector y or the sensitive attribute vector s_arr.

Apply your implementation on the Ricci dataset with default values for $\epsilon$ and $\Delta$, and compute the resulting CV score.

In [6]:
import numpy as np
from sklearn.naive_bayes import GaussianNB

class TwoNaiveBayes:
    
    # delta: adaptation rate (float)
    # eps: fairness tolerance, we consider a classification fair if the cv score on the training data is at least 1 - eps
    def __init__(self, delta=0.01, eps = 0.05):
        # your code here
        self.delta = delta
        self.esp = esp
        self.X = None
        self.y = None
        self.N = np.zeros((2,2))
        self.num_X = 0

    # X: 2d numerical numpy array reprenting the feature matrix without the sensitive attribute
    # s_arr: binary 1D numpy array representing the sensitive attribute
    # y: binary 1D numpy array representing the class labels to train on
    #
    # nothing is returned, only internal parameters are being fit
    def fit(self, X, s_arr, y):
        #your code here
        self.X = X
        self.y = y
        X = np.array(X)
        y = np.array(y)
        self.num_X = X.shape[0]
        num_pos_current = np.sum(y == 1)
        self.N = np.histogram2d(y, s, [2, 2], [[0, 2], [0, 2]])[0]
        for i in range(2):
            self.clr_[i].fit(X[s == i, :], y[s == i])
        
#         s_arr_new = np.insert(s_arr, 0, values=-1, axis=1)
    
        
       
        
    # X: 2d numerical numpy array reprenting the feature matrix without the sensitive attribute
    # s_arr: binary 1D numpy array representing the sensitive attribute to consider when predicting
    # 
    # return binary numpy array vector of fair class labels
    def predict(self,X,s_arr):
        # your code here

SyntaxError: unexpected EOF while parsing (<ipython-input-6-13096a935531>, line 31)

In [13]:

X = np.array([[1,2,3],
              [4,5,6],
              [7,8,9]])

# print(np.delete(X, 1, axis=1))
X_1 = np.insert(X, 0, values=-1, axis=1)
X_1

array([[-1,  1,  2,  3],
       [-1,  4,  5,  6],
       [-1,  7,  8,  9]])

###  Task 2: k-Means Clustering (1.5 pts)

We want to apply the $k$-means clustering algorithm from lecture to assign clusters to the dataset in "clusters.txt", which is loaded and plotted in the cell below.

In [ ]:
from matplotlib import pyplot as plt

X = np.loadtxt("clusters.txt")
plt.scatter(X[:,0],X[:,1], s=0.7)
plt.show()

#### a) Implementing k-means (0.75 pts)

Implement the $k$-means clustering algorithm using the function specification in the cell below.

In [ ]:
# INPUT VALUES:
# X: numerical 2D numpy array, each row is a sample
# k: int, number of clusters, ignored if init_points is not None
# init_points: list of row indices which indicate the data points that the clusters are initialized with
# -> default is None, indicating that random data points are initialized as cluster centers
# if specified, k is chosen as the number of cluster centers, and input for k is ignored
#
# RETURN:
# y: vector with cluster labels in {1,...,k}


def k_means(X, k=2, init_points = None):
    ## your code here

#### b) Application with random initialization (0.25 pts)

Apply your implementation 10 times on the data from clusters.txt, choosing $k = 4$ and random initalization of cluster centers. For each run, scatterplot the data as above, with different colors indicating the clusters.

#### c) k-means++ (0.5 pts)

Recall the initialization of cluster centers from the $k$-means++ algorithm as presented in lecture:

Let $X$ the set of data points, and $\mathcal{D}(x)$ denote the shortest distance from a data point to the closest center we have already chosen. Then, the k-means++ initialization consists of the following steps.
1. Take one center $c_1$, chosen uniformly at random from $X$.
2. Take a new center $c_i$, choosing $x \in X$ with probability $$p(x) = \frac{D(x)^2}{\sum_{x\in X} D(x)^2}.$$
3. Repeat Step 2 until we have taken $k$ centers altogether.

Implement this initialization, using the signature in the cell below. Use Euclidean distance to measure distance between any pair of points in the data.  
Apply this initialization to rerun the experiment from b). That is, run the $4$-means clustering algorithm ten times on the given data, using the initialization from k_means_pp, and scatterplot each clustering using different colors for different clusters. 

In [ ]:
# INPUT VALUES:
# X: numerical 2D numpy array, each row is a sample
# k: int, number of clusters, ignored if init_points is not None
#
# RETURN:
# init_points: list of row indices which indicate the data points that the clusters are initialized with

def init_k_means_pp(X,k):
    # your code here

### Task 3: Helping Santa (1.5 pts)

Santa Claus has gotten old and cannot deliver the christmas presents by himself anymore. Luckily, he has his team of 20 elves that are happy to carry all the presents out for him.
To distribute presents in an efficient way, he wants to partition the households to deliver presents to in an optimal way, by clustering the locations that the presents are delivered to.
A map with the coordinates of each household to deliver can be found in "xmas.txt", and it is loaded and plotted in the cell below.

In [ ]:
xmas = np.loadtxt("xmas.txt")
plt.scatter(xmas[:,0],xmas[:,1], s=0.2)
plt.show()

#### a) The Silhouette Coefficient (0.5 pts)

Since santa cannot judge the quality of a clustering by looking at a map, he needs to apply some quality measure that quantifies how good a clustering is.  

One such measure, which both evaluates how compact clusters are among themselves, and how well different clusters are separated, is the **silhouette coefficient**.
Assuming that all data points $x_1,\dots ,x_n$ are partitioned into $k$ clusters $C_1, \dots, C_k$, we can compute two coefficients for each data point $x_i$. On the one hand, assuming that $x_i \in C_l$,
$$
a(x_i) := \frac{\sum_{x_j\in C_l} d(x_i,x_j)}{|C_l| - 1}
$$
measures the compactness of the cluster $C_l$ to which $x_i$ belongs, and 
$$
b(x_i) := \min_{C_m\not= C_l}\left\{ \frac{\sum_{x_j\in C_m} d(x_i,x_j)}{|C_m|} \right\}
$$
captures the degree to which $x_i$ is separated from other clusters. Note that in these formulas, $d$ represents a distance function that can be chosen individually for the application at hand.
From these two values, we can compute the silhouette coefficient of $x_i$ via
$$
s(x_i) = \frac{b(x_i) - a(x_i)}{\max\{a(x_i),b(x_i)\}}.
$$
Note that the value of this coefficient lies between -1 and 1, with 1 being the optimum, and negative values indicating a bad clustering, where on expectation $x_i$ is further away from the data points in its own cluster than from points in any other cluster.  
To evaluate the quality of the **full clustering** of a dataset, one can then compute the **average silhouette coefficient of all points in a dataset**.  
Note that this measure does not require a fixed $k$, unlike the objective function in the k-means clustering algorithm, and thus it can also be used to find an optimal $k$.

Write a function that computes this silhouette score of a full clustering, using the _Euclidean distance_ as the distance function $d$. Use the signature which is specified in the cell below!

In [ ]:
# INPUT VALUES
# X: numerical 2D numpy array, each row is a data sample
# y: numpy vector with corresponding cluster labels
#
# return value: single float

def silhouette_score(X,y):
    # your code here

#### b) Finding the Optimal Hierarchical Clustering (1 pt)

On christmas eve, Santa wants to send an appropriate number of his elves out to deliver all the presents, and he wants to send at least 10 elves out, because the limited capacity of their sleighs does not allow for a smaller number. However, he is also fine with not sending all of his 20 elves out, since all the elves that are not delivering presents could prepare Santa's own traditional christmas party in the meantime.  
Thus, he decides to simply send out the number of elves between 10 and 20 that yields the best clustering with respect to the silhouette score. Since santa does not like the risks of random initializations, he wants to apply a hierarchical clustering algorithm for that task.

Find the optimal clustering of the households on the christmas map consisting of between 10 and 20 clusters, resulting from a hierarchical (agglomerative) clustering with either single, average or complete linkage, and using Euclidean distance. Save the corresponding vector of optimal cluster labels into a file "labels.npy" and submit this file along with your noteboook. Also provide all code that you used to find this optimum in the cell(s) below.    
Note that you do not need to implement hierarchical clustering yourself, but may resort to popular python libraries.
Also note that this task is not a class competition, it is only about applying the clustering algorithms and measures correctly.